In [3]:
import streamlit as st
import numpy as np

st.title("Binomial Option Pricing Model")

# User Inputs
S = st.number_input("Spot Price (S)", value=100.0)
K = st.number_input("Strike Price (K)", value=100.0)
T = st.number_input("Time to Maturity (T, in years)", value=1.0)
r = st.number_input("Risk-Free Rate (r, annualized)", value=0.05)
sigma = st.number_input("Volatility (σ)", value=0.2)
N = st.slider("Number of Steps (N)", min_value=1, max_value=200, value=3)
option_type = st.selectbox("Option Type", ("Call", "Put"))
style = st.selectbox("Style", ("European", "American"))

# Calculations
dt = T / N
u = np.exp(sigma * np.sqrt(dt))
d = 1 / u
p = (np.exp(r * dt) - d) / (u - d)
discount = np.exp(-r * dt)

# Build asset price tree
asset_prices = np.zeros((N+1, N+1))
asset_prices[0, 0] = S
for i in range(1, N+1):
    asset_prices[i, 0] = asset_prices[i-1, 0] * u
    for j in range(1, i+1):
        asset_prices[i, j] = asset_prices[i-1, j-1] * d

# Build option value tree
option_values = np.zeros_like(asset_prices)
for j in range(N+1):
    if option_type == "Call":
        option_values[N, j] = max(0, asset_prices[N, j] - K)
    else:
        option_values[N, j] = max(0, K - asset_prices[N, j])

# Backward induction
for i in range(N-1, -1, -1):
    for j in range(i+1):
        hold = discount * (p * option_values[i+1, j] + (1 - p) * option_values[i+1, j+1])
        if style == "American":
            if option_type == "Call":
                exercise = max(0, asset_prices[i, j] - K)
            else:
                exercise = max(0, K - asset_prices[i, j])
            option_values[i, j] = max(hold, exercise)
        else:
            option_values[i, j] = hold

st.subheader("Option Price")
st.success(f"{option_values[0,0]:.4f}")


2025-05-17 19:37:03.490 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 19:37:04.064 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-05-17 19:37:04.067 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 19:37:04.072 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 19:37:04.074 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 19:37:04.078 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 19:37:04.083 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-17 19:37:04.087 Session state does not 

DeltaGenerator()

In [4]:
import numpy as np

def binomial_option_pricing(S, K, T, r, sigma, N, option_type='call', style='european'):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    discount = np.exp(-r * dt)

    # Asset Price Tree
    asset_prices = np.zeros((N + 1, N + 1))
    asset_prices[0, 0] = S
    for i in range(1, N + 1):
        asset_prices[i, 0] = asset_prices[i - 1, 0] * u
        for j in range(1, i + 1):
            asset_prices[i, j] = asset_prices[i - 1, j - 1] * d

    # Option Value Tree
    option_values = np.zeros_like(asset_prices)
    for j in range(N + 1):
        if option_type == 'call':
            option_values[N, j] = max(0, asset_prices[N, j] - K)
        else:
            option_values[N, j] = max(0, K - asset_prices[N, j])

    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            hold = discount * (p * option_values[i + 1, j] + (1 - p) * option_values[i + 1, j + 1])
            if style == 'american':
                if option_type == 'call':
                    exercise = max(0, asset_prices[i, j] - K)
                else:
                    exercise = max(0, K - asset_prices[i, j])
                option_values[i, j] = max(hold, exercise)
            else:
                option_values[i, j] = hold

    return option_values[0, 0]


In [9]:
price = binomial_option_pricing(
    S=140,       # Spot price
    K=80,       # Strike price
    T=1/9,         # Time to maturity (in years)
    r=0.065,      # Risk-free interest rate
    sigma=0.68,   # Volatility
    N=6,         # Number of steps in the binomial tree
    option_type='call',    # 'call' or 'put'
    style='american'       # 'european' or 'american'
)

print("Option Price:", round(price, 4))


Option Price: 60.5757


In [13]:
%%writefile binomial_app.py
import streamlit as st
import numpy as np
import pandas as pd

st.title("📈 Binomial Option Pricing Calculator")

# Inputs
S = st.number_input("Spot Price (S)", value=100.0)
K = st.number_input("Strike Price (K)", value=100.0)
T = st.number_input("Time to Maturity (T, in years)", value=1.0)
r = st.number_input("Risk-Free Interest Rate (r)", value=0.05)
sigma = st.number_input("Volatility (σ)", value=0.2)
N = st.slider("Number of Steps (N)", min_value=1, max_value=100, value=3)
option_type = st.selectbox("Option Type", ("call", "put"))
style = st.selectbox("Option Style", ("european", "american"))

# Binomial Pricing Logic
def binomial_option_pricing(S, K, T, r, sigma, N, option_type='call', style='european'):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    discount = np.exp(-r * dt)

    # Asset Price Tree
    asset_prices = np.zeros((N + 1, N + 1))
    asset_prices[0, 0] = S
    for i in range(1, N + 1):
        asset_prices[i, 0] = asset_prices[i - 1, 0] * u
        for j in range(1, i + 1):
            asset_prices[i, j] = asset_prices[i - 1, j - 1] * d

    # Option Value Tree
    option_values = np.zeros_like(asset_prices)
    for j in range(N + 1):
        if option_type == 'call':
            option_values[N, j] = max(0, asset_prices[N, j] - K)
        else:
            option_values[N, j] = max(0, K - asset_prices[N, j])

    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            hold = discount * (p * option_values[i + 1, j] + (1 - p) * option_values[i + 1, j + 1])
            if style == 'american':
                if option_type == 'call':
                    exercise = max(0, asset_prices[i, j] - K)
                else:
                    exercise = max(0, K - asset_prices[i, j])
                option_values[i, j] = max(hold, exercise)
            else:
                option_values[i, j] = hold

    return option_values[0, 0], asset_prices, option_values

# Calculate and Display
price, asset_tree, option_tree = binomial_option_pricing(S, K, T, r, sigma, N, option_type, style)
st.subheader(f"📌 Option Price: `{price:.4f}`")

# Display Trees as Tables
asset_df = pd.DataFrame('', index=range(N+1), columns=range(N+1))
opt_df = pd.DataFrame('', index=range(N+1), columns=range(N+1))

for i in range(N+1):
    for j in range(i+1):
        asset_df.iloc[i, j] = f"{asset_tree[i, j]:.2f}"
        opt_df.iloc[i, j] = f"{option_tree[i, j]:.2f}"

st.write("📊 Asset Price Tree")
st.dataframe(asset_df)

st.write(f"📊 Option Value Tree ({style.title()} {option_type.title()})")
st.dataframe(opt_df)


Writing binomial_app.py


In [15]:
from google.colab import files
files.download('binomial_app.py')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00
